In [ ]:
import h5py,os,re,click
import numpy as np

ens='cB211.072.64'

path='data_aux/cfgs_run'
with open(path,'r') as f:
    cfgs=f.read().splitlines()

inpath=f'/p/project1/ngff/li47/code/scratch/run/05_moments_run5/{ens}/data_avgmore/'
outpath=f'/p/project1/ngff/li47/code/scratch/run/05_moments_run5/{ens}/data_merge/'
os.makedirs(outpath,exist_ok=True)

def run():
    files=os.listdir(inpath+cfgs[0])
    for ifile,file in enumerate(files):
        outfile=f'{outpath}/{file}'
        outfile_flag=outfile+'_flag'
        if os.path.isfile(outfile) and (not os.path.isfile(outfile_flag)):
            continue
        with open(outfile_flag,'w') as f:
            pass
        
        setupQ=True
        key2data={}
        for icfg,cfg in enumerate(cfgs):
            print(f'{ifile}/{len(files)}',f'{icfg}/{len(cfgs)}',end='               \r')
            infile=f'{inpath}/{cfg}/{file}'
            with h5py.File(infile) as f:
                if setupQ:
                    key2info={key:f[key][:] for key in f.keys() if key not in ['data']}
                    
                    key2info['notes']=[ele.decode() for ele in key2info['notes']]+['adding an additional dimension [cfg] in the beginning']
                    key2info['cfgs']=cfgs
                    
                    for key in f['data'].keys():
                        key2data[key]=[]
                        
                    setupQ=False
                    
                for key in f['data'].keys():
                    key2data[key].append(f[f'data/{key}'][:])
                
        with h5py.File(outfile,'w') as f:
            for key in key2info.keys():
                f.create_dataset(key,data=key2info[key])
            for key in key2data.keys():
                f.create_dataset(f'data/{key}',data=key2data[key])
                
        os.remove(outfile_flag)

run()